# Building agent prototype

This notebook covers how we can do prototyping with ai agent. This agent will have capabilities:

* To use postgres vector store we created earlier to get some information about the course.
* To call API server we deployed earlier to get information about the courses (name, price, etc) and to make payment url.

## Required Software Installed Locally

* Python version 3.9, 3.10, or 3.11. **Python3.12 will not work**.

* If you are using VSCode, please install Jupyter Notebook extensions.

* Jupyter notebook. Please follow this [installation guide](https://docs.jupyter.org/en/stable/install.html). You may choose whether you want to install classic jupyter notebook or jupyterlab (the next-gen web ui for jupyter)

    * [Classic jupyter notebook installation guide](https://docs.jupyter.org/en/stable/install/notebook-classic.html)

    * [Jupyterlab installation guide](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)

* Google Cloud CLI. Please follow this [installation guide](https://cloud.google.com/sdk/docs/install-sdk)

### Installing dependencies

In [1]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-aiplatform[langchain]
google-cloud-aiplatform[reasoningengine]
langchain
langchain_core
langchain_community
langchain-google-vertexai==2.0.8
cloudpickle
pydantic==2.9.2
langchain-google-community
google-cloud-discoveryengine
nest-asyncio
asyncio==3.4.3
asyncpg==0.29.0
cloud-sql-python-connector[asyncpg]
langchain-google-cloud-sql-pg
numpy
pandas
pgvector
psycopg2-binary
langchain-openai
langgraph
traceloop-sdk
opentelemetry-instrumentation-google-generativeai
opentelemetry-instrumentation-langchain
opentelemetry-instrumentation-vertexai
python-dotenv

Overwriting requirements.txt


In [2]:
!pip install --upgrade -r requirements.txt

  Using cached numpy-2.2.2-cp310-cp310-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached cloud_sql_python_connector-1.16.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of cloud-sql-python-connector[asyncpg] to determine which version is compatible with other requirements. This could take a while.
  Using cached cloud_sql_python_connector-1.15.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached cloud_sql_python_connector-1.14.0-py2.py3-none-any.whl.metadata (28 kB)

[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


## Setting up Google Cloud Account

#### Recommended account setup

if you are running this in jupyter notebook locally, you may need to login to google cloud by running the following command from terminal:

```
gcloud auth login
gcloud auth application-default login
```

If you are using Google Colabs, you need to authenticate with your google account by running the following notebook cell. 

> Please remember that you will need to do this on each jupyter notebook during this workshop

In [ ]:
# #@markdown ###Authenticate your Google Cloud Account and enable APIs.
# # Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

Let's start with importhing few stuff:

In [3]:
from IPython.display import display, Markdown

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field

from langchain.memory import ChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

import pandas as pd
from vertexai.preview import reasoning_engines
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
import requests

/Users/pabrik/working/academic/MASTER/PROJECTS/InformationRetrieval/aiagent-from-scratch/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Let's define some variable. Please update these following variables according to your setup:
* `project_id`
* `region`
* `staging_bucket_name`
* `instance_name`, `database_password`, `database_name`, `database_user`

In [4]:
project_id = "gen-lang-client-0521448746"  # @param {type:"string"}
region = "us-central1" #change this to project location
staging_bucket_name = "devfest24-demo-bucket" # @param {type:"string"} #change this with your staging bucket name
instance_name="devfest24-demo" # @param {type:"string"}
database_password = 'testing' # @param {type:"string"}
database_name = 'testing' # @param {type:"string"}
database_user = 'testing' # @param {type:"string"}

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

# dont update variable below

!gcloud config set project {project_id} --quiet

cloudrun_services = !gcloud run services describe courses-api --region=us-central1 --format='value(status.url)'
api_base_url = cloudrun_services[0]

staging_bucket_uri = f"gs://{staging_bucket_name}"
# get the ip address of the cloudsql instance
ip_addresses = !gcloud sql instances describe {instance_name} --format="value(ipAddresses[0].ipAddress)"
database_host = ip_addresses[0]

gemini_embedding_model = "text-embedding-004"
gemini_llm_model = "gemini-1.5-pro"
embeddings_table_name = "course_content_embeddings"
chat_history_table_name = "chat_histories"

print(f"API Base URL: {api_base_url}")
print(f"Database Host: {database_host}")


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].
API Base URL: https://courses-api-guckng3ccq-uc.a.run.app
Database Host: 34.42.192.52


Let's initialize vertex ai, postgres engine and vector store. This is very similar to the previous module. But instead of using this directly, we are going to use it in Langchain Tool to add capability to the agent:

In [5]:
import vertexai
vertexai.init(project=project_id, location=region, staging_bucket=staging_bucket_uri)

from langchain_google_vertexai import VertexAIEmbeddings
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)

from langchain_google_cloud_sql_pg import PostgresEngine

pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user=database_password,
    password=database_password,
)

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresVectorStore

sample_vector_table_name = "course_content_embeddings"

vector_store = PostgresVectorStore.create_sync(
            pg_engine,
            table_name=embeddings_table_name,
            embedding_service=embeddings_service,
        )
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

## Langchain Tool

### Search course tool with Postgres Vector Store

This is the first tool that we will create. It is used to search content from the database given a user query.

If you see internally, it only call `retriever.invoke()` and return the value. The other important thing is the description of the function. Thats how the agent knows when it needs to use this tool

In [6]:
@tool
def search_course_content(query: str) -> str:
    """Explain about software security course materials."""
    result = str(retriever.invoke(query))
    return result

In [7]:
search_course_content.invoke("best practices for forgot password") 

'[Document(metadata={\'course_content_id\': \'a2a2096f-1cb2-43c4-9f26-ba53c321c7ba\', \'title\': \'Forgot Password Cheat Sheet\'}, page_content=\'The following short guidelines can be used as a quick reference to protect the forgot password service:\\n\\n- **Return a consistent message for both existent and non-existent accounts.**\\n- **Ensure that the time taken for the user response message is uniform.**\\n- **Use a side-channel to communicate the method to reset their password.**\\n- **Use [URL tokens](#url-tokens) for the simplest and fastest implementation.**\\n- **Ensure that generated tokens or codes are:**\\n    - **Randomly generated using a cryptographically safe algorithm.**\\n    - **Sufficiently long to protect against brute-force attacks.**\\n    - **Stored securely.**\\n    - **Single use and expire after an appropriate period.**\\n- **Do not make a change to the account until a valid token is presented, such as locking out the account**\'), Document(metadata={\'course_

### Creating tool which calls API

Now we define a simple python api client which will call the api we deployed earlier to cloud run.

In [8]:
class CourseAPIClient:
  def __init__(self, url=api_base_url):
    self.url = url
    
  def list_courses(self):
      response = requests.get(f"{self.url}/courses")
      return response.json()

  def get_course(self, course_name):
      response = requests.get(f"{self.url}/courses/{course_name}")
      return response.json()

  def create_order(self, course, user_name, user_email):
      payload = {
          "course": course,
          "user_name": user_name,
          "user_email": user_email
      }
      response = requests.post(f"{self.url}/orders", json=payload)
      return response.json()

  def get_order(self, order_id):
      response = requests.get(f"{self.url}/orders/{order_id}")
      return response.json()

  def pay_order(self, order_id):
      response = requests.post(f"{self.url}/orders/{order_id}:pay")
      return response.json()

  def get_payment_page_url(self, order_id):
      return f"{self.url}/orders/{order_id}/payment"

For each api, we will define the tools here and call the relevant api function. Please note the description as well.

In [9]:
from typing import List

@tool
def list_courses() -> List[str]:
  """List all available courses sold on the platform."""
  client = CourseAPIClient()
  return client.list_courses()

To help the agent decide what should be the input of the function, we can also define a input class and give proper description for the function and each input arguments

In [10]:
class GetCourseInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")

@tool("get-course-tool", args_schema=GetCourseInput)
def get_course(course: str) -> str:
  """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
  This function can be used to get course details such as course price, etc."""
  client = CourseAPIClient()
  return client.get_course(course)

Here you may use multiple arguments and perform some computation within the function/tools. In this case, the tool is used to create the order and return the order id and link to make the payment

In [11]:
class CreateOrderInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")
    user_name: str = Field(description="name of the user who is purchasing the course .")
    user_email: str = Field(description="email of the user who is purchasing the course.")

@tool("create-order-tool", args_schema=CreateOrderInput)
def create_order(course: str, user_name: str, user_email: str) -> str:
  """Create order for a course. This function can be used to create an order for a course. When this function returns successfully, it will return payment url to user to make payment. """
  client = CourseAPIClient()
  
  print(f"Creating order for course: {course}, user_name: {user_name}, user_email: {user_email}")
  
  res = client.create_order(course, user_name, user_email)
  print(res)
  order_id = res["order_id"]
  payment_url = f"{api_base_url}/orders/{order_id}/payment"
  return f"Order number {order_id} created successfully. Payment URL: {payment_url}"

In [12]:
create_order.invoke({"course":"software-security", "user_name":"John Doe", "user_email":"imre@gmail.com"}) 

Creating order for course: software-security, user_name: John Doe, user_email: imre@gmail.com
{'order_id': '7a4e89d0-a678-49a0-ac69-37a33beaeff1'}


'Order number 7a4e89d0-a678-49a0-ac69-37a33beaeff1 created successfully. Payment URL: https://courses-api-guckng3ccq-uc.a.run.app/orders/7a4e89d0-a678-49a0-ac69-37a33beaeff1/payment'

In [13]:
class GetOrderInput(BaseModel):
    order_number: str = Field(description="order number identifier. this is a unique identifier in uuid format.")

@tool("get-order-tool", args_schema=GetOrderInput)
def get_order(order_number: str) -> str:
  """Get order by using order number. This function can be used to get order details such as payment status to check whether the order has been paid or not. If user already paid the course, say thanks"""
  client = CourseAPIClient()
  return client.get_order(order_number)

Once we have the tools ready, we are going to put them into an array which will be used later

In [14]:
tools = [search_course_content, list_courses, get_course, create_order, get_order]

### Prompt

This is prompt that we are going to use. On the prompt below, we defined few things:
* System context. This is used to tell who the bot is and what it should and shouldn't do.
* Adding chat history. This is used so that the agent can keep the conversation relevant and stays within the same context.
* User query. This is query or question directly given by the user
* Agent scratchpad. This is internal data used by the agent to decide which tools to use.

In [15]:
prompt = {
    "chat_history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (
        lambda x: format_to_openai_function_messages(x["intermediate_steps"])
    ),
} | ChatPromptTemplate(
  messages = [
    SystemMessagePromptTemplate.from_template("""
      You are a bot assistant that sells online course about software security. You only use information provided from datastore or tools. You can provide the information that is relevant to the user's question or the summary of the content. If they ask about the content, you can give them more detail about the content. If the user seems interested, you may suggest the user to enroll in the course. 
      """),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template("Use tools to answer this questions: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
  ]
)

### Message History

To keep the conversation on context, we use chat history storage in PostgresSQL. This is used to store the conversation history between the user and the agent. This is used to keep the conversation relevant and to keep the context of the conversation.

In [16]:
from langchain_google_cloud_sql_pg import PostgresChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return PostgresChatMessageHistory.create_sync(
        pg_engine,
        table_name=chat_history_table_name,
        session_id=session_id,
    )

### Defining Agent

This is where we define configuration for the agent. 

Here we defined:
* Safety settings for Gemini
* Model parameter (e.g. temperature and safety settings)
* Agent creation where we add the tools, promopt, model, session history, etc


In [17]:
## Model safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

## Model parameters
model_kwargs = {
    "temperature": 0.5,
    "safety_settings": safety_settings,
}

agent = reasoning_engines.LangchainAgent(
    model=gemini_llm_model,
    tools=tools,
    prompt=prompt,    
    chat_history=get_session_history,
    agent_executor_kwargs={
      "return_intermediate_steps": True,
    },
    model_kwargs=model_kwargs,
    enable_tracing=True,
)

### Testing the agent

In [18]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

Generated session ID: 2a586200-b144-46b8-ad65-7a5451827def


In [19]:
response = agent.query(
  input="Can you please share what are being taught on this course?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

This course covers a wide range of software security topics, including authorization, REST API security, and file upload security. 

Let me give you some more details about what you'll learn:

**Authorization:** You'll understand how to design and implement robust authorization mechanisms. The course covers topics like:
* Ensuring that access control policies are applied to static resources.
* Creating unit and integration test cases for your authorization logic.
* Understanding and implementing concepts like Least Privilege, Role-Based Access Control (RBAC), and Attribute-Based Access Control (ABAC).

**REST Security:** You'll learn how to secure your REST APIs, covering:
* Implementing state management securely in a stateless manner.
* Utilizing standard HTTP verbs and error codes effectively.
* Understanding and leveraging HATEOAS (Hypermedia As The Engine of Application State).
* Implementing HTTPS for secure communication and authentication.

**File Upload Security:** You'll discover how to handle file uploads securely, including:
* Setting appropriate upload and download limits to prevent abuse.
* Securely extracting and processing uploaded files to mitigate risks.

This is just a glimpse of what's covered.  Are you interested in learning more about any of these topics or perhaps enrolling in the course? 


In [21]:
response = agent.query(
  input="Does it teach about how to design a forgot password system securely?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

Yes, the course includes a section dedicated to designing secure forgot password systems. It covers best practices and common pitfalls to avoid, such as:

* **Protecting against account enumeration:** The system should not reveal whether an account exists based on the forgot password request.
* **Secure token generation and handling:**  Using cryptographically secure random tokens with appropriate expiration times.
* **Rate limiting:** Implementing measures to prevent brute-force attacks. 
* **Multi-factor authentication (MFA) recovery:** Providing secure ways for users to recover their accounts even if they lose access to their MFA methods.

Would you like me to elaborate on any of these topics or other aspects of secure forgot password system design? 


In [22]:
response = agent.query(
  input="How much this course costs?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

The "Software Security" course costs $100 USD. Would you like to enroll? 


In [23]:
response = agent.query(
  input="Yes. I want to enroll. What information do you need?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

To enroll you in the "Software Security" course, please provide me with your name and email address. 


In [24]:
response = agent.query(
  input="My name is Mulyono, and my email is fufufafa@gmail.com",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

Span has more then 32 attributes, some will be truncated


Creating order for course: software-security, user_name: Mulyono, user_email: fufufafa@gmail.com
{'order_id': '3a76b76e-0918-4263-b87b-0cffb7d9a854'}


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Span has more then 32 attributes, some will be truncated


Thank you, Mulyono. Your order for the "Software Security" course has been created. 

Your order number is 3a76b76e-0918-4263-b87b-0cffb7d9a854. 

To proceed with the payment, please visit this URL: https://courses-api-guckng3ccq-uc.a.run.app/orders/3a76b76e-0918-4263-b87b-0cffb7d9a854/payment 

You'll receive a confirmation email once your payment is processed. 


In [25]:
response = agent.query(
  input="I have made the payment. Can you please check?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

Span has more then 32 attributes, some will be truncated
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds 

Thank you for confirming your payment. Your order is now confirmed. You now have access to all the course materials. Happy learning! 


In [27]:
response = agent.query(
  input="kamu harus berbahasa indonesia, saya sudah memiliki course apa saja?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

Span has more then 32 attributes, some will be truncated


Anda terdaftar di kursus "Software Security". Selamat belajar! 


In [28]:
response = agent.query(
  input="materi apa saja yang terdapat pada course?",
  config={"configurable": {"session_id": session_id}},
)
display(Markdown(response["output"]))

Span has more then 32 attributes, some will be truncated
Span has more then 32 attributes, some will be truncated


The "Software Security" course contains a cheat sheet about authorization. It covers topics like enforcing least privileges, thoroughly reviewing the authorization logic of chosen tools and technologies, and implementing appropriate logging. 

Would you like to know more about any of these topics? 
